In [1]:
import pandas as pd
import numpy as np

In [ ]:
from telegram_notifier import send_message as telegram_bot_sendtext

In [2]:
import torch
print(f"Torch Version: {torch.__version__}")

import transformers
print(f"transformers (Adapter) Version: {transformers.__version__}")

Torch Version: 1.8.1
transformers (Adapter) Version: 2.0.1


In [3]:
from transformers import RobertaTokenizer
import numpy as np

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=80, truncation=True, padding="max_length")

In [4]:
from ner_dataset import get_trainset_data_loader

all_tags, trainset, trainloader = get_trainset_data_loader(tokenizer, BATCH_SIZE=128)

labels: ['B-art' 'B-eve' 'B-geo' 'B-gpe' 'B-nat' 'B-org' 'B-per' 'B-tim'
 'CountryCode' 'CryptoCurrencyCode' 'CurrencyCode' 'Event' 'Float' 'I-art'
 'I-eve' 'I-geo' 'I-gpe' 'I-nat' 'I-org' 'I-per' 'I-tim' 'Integer'
 'Location' 'Month' 'O' 'Object' 'Party' 'Race' 'SpecialTerm'
 'TemporalUnit' 'Time' 'Timezone' 'US_States']


In [5]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=len(all_tags),
    label2id = trainset.label_map, 
    id2label = trainset.id2label
)

name = model.load_adapter("./save_adapters/ALL_tag_0730")
model.add_tagging_head(
        name,
        num_labels=len(trainset.label_map.keys()), overwrite_ok=True
      )
model.train_adapter(name)

In [7]:
device_id = 0
device = torch.device(f"cuda:{device_id}" if torch.cuda.is_available() else "cpu")

In [8]:
all_tags = ['Float','TemporalUnit','I-gpe','CountryCode','CurrencyCode','Timezone','CryptoCurrencyCode','Month','Party','B-tim','I-art','Time','B-per','B-gpe','B-geo','O','Location','Event','I-nat','Race','B-org','I-geo','I-tim','I-eve','SpecialTerm','B-art','US_States','B-eve','I-org','B-nat','Object','I-per','Integer']

In [ ]:
for index, tag in enumerate(all_tags):
    if index % 2 == device_id:
        print(f"\nSkip {tag}.\n")
        continue
    model = RobertaModelWithHeads.from_pretrained(
        "roberta-base",
        config=config,
        )


    try:
        model.add_adapter(tag)
        model.add_tagging_head(
            tag,
            num_labels=1
          )
    except: pass
    model.train_adapter(tag)
    model = model.to(device)
    
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
                    {
                        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                        "weight_decay": 1e-5,
                    },
                    {
                        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                        "weight_decay": 0.0,
                    },
                ]
    optimizer = torch.optim.AdamW(params=optimizer_grouped_parameters, lr=1e-4)
    
    for epoch in range(4):
        print(f"\n{tag}: epoch {epoch}")
        for i, data in enumerate(trainloader):

            tokens_tensors, segments_tensors, \
            masks_tensors, labels = [t.to(device) for t in data]

            outputs = model(input_ids = tokens_tensors,
                attention_mask=masks_tensors,
                token_type_ids=segments_tensors)


            logits = outputs[0]

            current_label = labels.view(-1, labels.shape[-1])[:, trainset.label_map[tag]]
            current_label = current_label.view(-1)

            active_logits = logits.view(-1, logits.shape[-1])[masks_tensors.view(-1) == 1]
            active_labels = current_label[masks_tensors.view(-1)== 1]

            actual = current_label[masks_tensors.view(-1)== 1].float().view(-1,1)
            """

            actual = torch.ones(active_logits.shape, device = device)

            actual[:, 0] = (active_labels == 0).long()
            actual[:, 1] = (active_labels == 1).long()"""


            loss_fct = torch.nn.BCEWithLogitsLoss()

            loss = loss_fct(active_logits, actual)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            if i % 100 == 0:
                print(f"\tLoss: {loss}")
        telegram_bot_sendtext(f"\n{tag}: epoch {epoch}, loss = {loss}")
        filename = f"{tag}_epoch_{epoch}_0731"
        model.save_adapter(f"./save_adapters/{filename}", model.active_adapters[0])
        model.save_head(f"./save_heads/{filename}", model.active_head)
    filename = f"{tag}_0731"
    model.save_adapter(f"./save_adapters/{filename}", model.active_adapters[0])
    model.save_head(f"./save_heads/{filename}", model.active_head)


Skip Float.



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere


TemporalUnit: epoch 0
	Loss: 0.7386174201965332
	Loss: 0.07240715622901917
	Loss: 0.04146174341440201
	Loss: 0.03204631432890892

TemporalUnit: epoch 1
	Loss: 0.012723155319690704
	Loss: 0.014457549899816513
	Loss: 0.026513006538152695
	Loss: 0.019395049661397934

TemporalUnit: epoch 2
	Loss: 0.008143000304698944
	Loss: 0.007894353941082954
	Loss: 0.025094568729400635
	Loss: 0.014674236066639423

TemporalUnit: epoch 3
	Loss: 0.00591354351490736
	Loss: 0.005637414287775755
	Loss: 0.023329773917794228
	Loss: 0.008922210894525051

Skip I-gpe.



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids', 'roberta.encoder.layer.0.output.adapters.TemporalUnit.adapter_down.0.weight', 'roberta.encoder.layer.0.out


CountryCode: epoch 0
	Loss: 0.572617769241333
	Loss: 0.008682194165885448
	Loss: 0.002880189334973693
	Loss: 0.0003081628819927573
	Loss: 0.008424567058682442
	Loss: 0.003085791366174817
	Loss: 0.00029761993209831417

CountryCode: epoch 2
	Loss: 0.0001814014685805887
	Loss: 0.0029888416174799204
	Loss: 0.00038245366886258125
	Loss: 2.5794954126467928e-05

CountryCode: epoch 3
	Loss: 0.0002682531194295734
	Loss: 0.0004629404575098306
	Loss: 0.0001749052753439173
	Loss: 2.4363498596358113e-05

Skip CurrencyCode.



...


B-eve: epoch 0
	Loss: 0.7118805646896362
	Loss: 0.003172032767906785
	Loss: 0.00028799724532291293
	Loss: 0.003089305479079485

B-eve: epoch 1
	Loss: 0.00026710834936238825
	Loss: 0.002940287347882986
	Loss: 0.0003255342599004507
	Loss: 0.003120912006124854

B-eve: epoch 2
	Loss: 0.00019505243108142167
	Loss: 0.0030227871611714363
	Loss: 0.0002776059845928103
	Loss: 0.0010170197347179055

B-eve: epoch 3
	Loss: 0.00016015704022720456
	Loss: 0.002065422013401985
	Loss: 0.00016677241364959627
	Loss: 0.0007221006671898067

Skip I-org.



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids', 'roberta.encoder.layer.0.output.adapters.TemporalUnit.adapter_down.0.weight', 'roberta.encoder.layer.0.out


B-nat: epoch 0
	Loss: 0.7147560119628906
	Loss: 0.0002648832742124796
	Loss: 0.00015125941717997193
	Loss: 0.0002162465243600309

B-nat: epoch 1
	Loss: 0.002944048959761858
	Loss: 0.00016447629604954273
	Loss: 0.00014111264317762107
	Loss: 0.00023127629538066685

B-nat: epoch 2
	Loss: 0.0029876583721488714
	Loss: 0.00015380010881926864
	Loss: 0.00014603258750867099
	Loss: 0.00025136873591691256

B-nat: epoch 3
	Loss: 0.002940491307526827
	Loss: 0.0001735145051497966
	Loss: 0.00014997864491306245
	Loss: 0.00029573807842098176

Skip Object.



Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids', 'roberta.encoder.layer.0.output.adapters.TemporalUnit.adapter_down.0.weight', 'roberta.encoder.layer.0.out


I-per: epoch 0
	Loss: 0.7242643237113953
	Loss: 0.08145652711391449
	Loss: 0.055750615894794464
	Loss: 0.022875530645251274

I-per: epoch 1
	Loss: 0.030850324779748917
	Loss: 0.025137191638350487
	Loss: 0.02453632839024067
	Loss: 0.010009382851421833

I-per: epoch 2
	Loss: 0.020128754898905754
	Loss: 0.01920315809547901
	Loss: 0.020762773230671883
	Loss: 0.006889167707413435

I-per: epoch 3
	Loss: 0.01890278421342373
	Loss: 0.014045740477740765
	Loss: 0.021859019994735718
	Loss: 0.005650461185723543


In [ ]:
label_id_mapping = trainset.label_map

id_label_mapping = dict()
for key in label_id_mapping.keys():
    id_label_mapping[label_id_mapping[key]] = key

def test_model(model, sentence, device = "cpu"):
    tokenized_sentence = torch.tensor([tokenizer.encode(sentence)])
    pos = torch.tensor([[0] * len(tokenized_sentence)])
    tags = torch.tensor([[1] * len(tokenized_sentence)])

    model = model.to(device)
    outputs = model(input_ids=tokenized_sentence.to(device), 
                    token_type_ids=pos.to(device), 
                    attention_mask=tags.to(device))

    logits = outputs[0]

    _, pred_labels = torch.max(logits, 2)

    out_labels = []
    for row in pred_labels:
        result = list(map(lambda x: id_label_mapping[int(x)], row))
        out_labels.append(result)
    #return tokenizer.tokenize(sentence), out_labels[0], logits
    return tokenizer.tokenize(sentence), out_labels[0][1:-1], logits[:, 1:-1]

In [ ]:
sentence = "Dan will be deemed to have completed its delivery obligations before 2021-7-5 if in Niall's opinion, the Jeep Car satisfies the Acceptance Criteria, and Niall notifies Dan in writing that it is accepting the Jeep Car."

In [ ]:
tokenized_sentence = torch.tensor([tokenizer.encode(sentence)])
pos = torch.tensor([[0] * len(tokenized_sentence)])
tags = torch.tensor([[1] * len(tokenized_sentence)])

model = model.to(device)
outputs = model(input_ids=tokenized_sentence.to(device), 
                token_type_ids=pos.to(device), 
                attention_mask=tags.to(device))

In [ ]:
for i, text in enumerate(tokenizer.tokenize(sentence)):
    print(f"{text}: {outputs[0].view(-1)[i]}")

In [ ]:
sentence = "Dan Will be deemed to have completed its delivery obligations before 2021-7-5 if in Niall's opinion, the Jeep Car satisfies the Acceptance Criteria, and Niall notifies Dan in writing that it is accepting the Jeep Car."
sen, pred, logits = test_model(model, sentence, device = 'cpu')

In [ ]:
a = tokenizer.tokenize(sentence)[1]

In [ ]:
np.array(sen)

In [ ]:
np.array(pred)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
def interact_word(i):
    print(i)
    print(sen[i])
    target = out[i]

    for i in range(len(target)):
        print(f"{i} {id_label_mapping[i].ljust(6)} \t: {target[i]:.5f}")

In [ ]:
out = logits[0]
interact(lambda x: interact_word(x), x=widgets.IntSlider(min=0, max=len(sen)-1, step=1, value=0))

In [ ]:
print("OK")